In [11]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import folium
import pandas as pd
import geopandas as gpd
from dash.dependencies import Input, Output
import io
import base64

# Load data
df = pd.read_csv('co2_emission.csv')
df = df.rename(columns={'Annual CO₂ emissions (tonnes )': 'CO2_emission'})
world_geojson = gpd.read_file('countries.geo.json')

# Function to update map based on selected year
def update_map(year):
    year_data = df[df['Year'] == year]
    merged_data = world_geojson.merge(year_data, left_on='name', right_on='Entity', how='left')
    
    # Create the map
    m = folium.Map(location=[20, 0], zoom_start=2)
    
    folium.Choropleth(
        geo_data=world_geojson,
        name='CO2 Emissions',
        data=merged_data,
        columns=['name', 'CO2_emission'],
        key_on='feature.properties.name',
        fill_color='YlGnBu',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='CO₂ Emissions (tonnes)',
    ).add_to(m)
    
    # Save map to a BytesIO buffer
    map_io = io.BytesIO()
    m.save(map_io, close_file=False)
    map_io.seek(0)
    
    # Encode the map as base64
    map_base64 = base64.b64encode(map_io.read()).decode('utf-8')
    
    # Return the base64 string to embed in an iframe
    return f"data:text/html;base64,{map_base64}"

# Create Dash app
app = dash.Dash(__name__)

# Layout of the web page
app.layout = html.Div([
    html.H1('CO₂ Emission Map'),
    dcc.Slider(
        id='year-slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=2015,
        step=1,
    ),
    html.Div(id='map-container')
])

# Update the map when slider changes
@app.callback(
    Output('map-container', 'children'),
    [Input('year-slider', 'value')]
)
def update_map_callback(year):
    map_html = update_map(year)
    return html.Iframe(src=map_html, width='100%', height='600')

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
